# PG-AQO Workload Evaluation

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

## Data loading

Run 1:

In [2]:
df1 = pd.read_csv("workloads/job-complete-aqo-1-cout.csv")
df1.reset_index(inplace=True)
df1["index"] += 1
df1.sort_values(by="label", inplace=True)

Run 2:

In [3]:
df2 = pd.read_csv("workloads/job-complete-aqo-2-cout.csv")
df2.reset_index(inplace=True)
df2["index"] += 1
df2.sort_values(by="label", inplace=True)

Run 3:

In [4]:
df3 = pd.read_csv("workloads/job-complete-aqo-3-cout.csv")
df3.reset_index(inplace=True)
df3["index"] += 1
df3.sort_values(by="label", inplace=True)

## Detecting queries with different $C_{out}$ values

In [5]:
proj = ["cout", "t_exec", "label", "index"]
_m1 = pd.merge(df1[proj], df2[proj], on="label", suffixes=["_run1", "_run2"])
df_comb = pd.merge(_m1, df3[proj], on="label", suffixes=["", "_run3"])
df_comb.set_index("label", inplace=True)
df_comb.rename(columns={"cout": "cout_run3", "t_exec": "t_exec_run3", "index": "index_run3"}, inplace=True)

In [6]:
cout_cols = ["cout_run1", "index_run1", "cout_run2", "index_run2", "cout_run3", "index_run3"]
t_exec_cols = ["t_exec_run1", "index_run1", "t_exec_run2", "index_run2", "t_exec_run3", "index_run3"]

In [7]:
df_comb["diff_1_2"] = df_comb.cout_run1 != df_comb.cout_run2
df_comb["diff_1_3"] = df_comb.cout_run1 != df_comb.cout_run3
df_comb["diff_2_3"] = df_comb.cout_run2 != df_comb.cout_run3
df_comb["diff_any"] = df_comb.diff_1_2 | df_comb.diff_1_3 | df_comb.diff_2_3
df_comb[df_comb.diff_any][cout_cols]

,cout_run1,index_run1,cout_run2,index_run2,cout_run3,index_run3
label,,,,,,
13b,1912702,50,1912702,99,2641491,37
13c,2640445,26,2640445,72,1912521,108
16a,2602623,108,14366,26,14366,73
16d,216884,43,619573,50,619573,98
17b,399376,30,3599101,108,399376,70
17c,3533427,83,362078,55,3533427,113
22a,167972,101,492850,96,167972,32
22b,380673,25,163944,103,380673,26
2a,68686,41,804167,59,68546,107


In [8]:
df_comb[df_comb.diff_any][t_exec_cols]

,t_exec_run1,index_run1,t_exec_run2,index_run2,t_exec_run3,index_run3
label,,,,,,
13b,14931.466,50,15476.340,99,20618.351,37
13c,22531.254,26,20874.950,72,14728.999,108
16a,24687.911,108,14697.899,26,14672.818,73
16d,20714.552,43,20028.361,50,19205.697,98
17b,30549.297,30,27558.473,108,25974.502,70
17c,25371.218,83,23435.039,55,25159.056,113
22a,10055.220,101,11210.206,96,9831.744,32
22b,11267.370,25,10086.261,103,10638.097,26
2a,2575.391,41,5760.084,59,2319.825,107
